# 1. 導入套件

In [1]:
# 主要套件
import torch

# 資料處理
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# 建立機器模型
from torch import nn
from torch import optim

# 訓練過程視覺化
from torch.utils.tensorboard import SummaryWriter

# 數學運算
import math

# 2. 修正參數

In [2]:
# 擷取資料檔案
gesture_data_file = "../dataset/train_data-x_y.csv"

In [3]:
# 取得 GPU 或是 CPU 的設備進行訓練
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
# 批次
batch_size = 64

In [5]:
# 優化函數
optim_fn = optim.SGD
# 損失函數
loss_fn = nn.CrossEntropyLoss()

In [6]:
# 學習率
learning_rate = 1e-5
# 世代
epochs = 60

In [7]:
# TensorBoard 的物件
writer = SummaryWriter()

# 3. 函數

In [8]:
# 訓練函數
def train(dataloader:DataLoader, model:nn.Module, loss_fn:torch.nn.modules.Module, optimizer:optim.Optimizer, debug:bool=False):
    size = len(dataloader.dataset)
    
    # 開始訓練
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 轉換成可讓模型訓練的格式
        X, y = X.to(device), y.to(device)
        
        # 計算預測誤差
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation 反向傳播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 顯示訓練輸出
        if debug:
            if batch % 100 == 0:
                loss, current = loss.item(), (batch+1)*len(X)
                print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [9]:
# 測試函數
# 回傳成功率，測試 loss
def test(dataloader:DataLoader, model:nn.Module, loss_fn:torch.nn.modules.Module, debug:bool=False):
    
    # 計算結果參數
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # 測試誤差、正確率
    test_loss, correct = 0, 0
    
    # 開始驗證
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            # 轉換成可讓模型訓練的格式
            X, y = X.to(device), y.to(device)

            # 計算預測誤差
            pred = model(X)
            loss = loss_fn(pred, y)
            
            # 計算誤差、正確率
            test_loss += loss.item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    # 計算 loss 和 correct
    test_loss /= num_batches
    correct /= size
    
    # 顯示驗證輸出
    if debug:
        print(f"Test Error: \n Accuracy: {(100*correct):0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    return correct*100, test_loss

In [87]:
# 計算關鍵點的夾角
def angle_between_points(p0:[int, int], p1:[int, int], p2:[int, int]):
    angle = math.degrees(
        math.atan2(p2[1] - p1[1], p2[0] - p1[0]) -
        math.atan2(p0[1] - p1[1], p0[0] - p1[0])
    )
    
    if angle < 0:
        angle = -angle

    if angle > 180:
        angle -= 180

    return angle

In [11]:
# 輔助函式
from playsound import playsound
# 撥放音樂
def PlaySound():
    sound_pth = "E:\Media Cabinet\Musics\Musics\dio zawaruto.mp3"
    playsound(sound_pth)

# 4. 類別

In [63]:
data = pd.read_csv(gesture_data_file)

COCO 的姿態關鍵點  
![COCO 的姿態關鍵點](COCO_Keypoints,_Used_in_MoveNet_and_PoseNet.png)

In [83]:
# 姿態資料集
class GestureDataset(Dataset):
    def __init__(self, df:pd.DataFrame):
        self.dataframe = df
        
        result = self.dataframe.iloc[:,-1]
        
        # 計算 17 個關鍵點在檔案中的位置
        p = [None]*17
        for i in range(17):
            p[i] = i*2
        
        # 初始化各關節點角度
        size = len(df)
        
        left_elbow_angle = [None] * size
        right_elbow_angle = [None] * size
        
        left_arm_angle = [None] * size
        right_arm_angle = [None] * size
        
        left_hip_angle = [None] * size
        right_hip_angle = [None] * size
        
        left_knee_angle = [None] * size
        right_knee_angle = [None] * size
        
        # 計算各關節點角度
        for i in range(size):
            left_elbow_angle[i] = angle_between_points(
                p0=data.iloc[i, p[5]:p[5]+2], # p9
                p1=data.iloc[i, p[7]:p[7]+2], # p7
                p2=data.iloc[i, p[9]:p[9]+2]  # p5
            )
            right_elbow_angle[i] = angle_between_points(
                p0=data.iloc[i, p[6]:p[6]+2], # p6
                p1=data.iloc[i, p[8]:p[8]+2], # p8
                p2=data.iloc[i, p[10]:p[10]+2]  # p10
            )
            
            left_arm_angle[i] = angle_between_points(
                p0=data.iloc[i, p[7]:p[7]+2], # p7
                p1=data.iloc[i, p[5]:p[5]+2], # p5
                p2=data.iloc[i, p[11]:p[11]+2]  # p11
            )
            right_arm_angle[i] = angle_between_points(
                p0=data.iloc[i, p[8]:p[8]+2], # p8
                p1=data.iloc[i, p[6]:p[6]+2], # p6
                p2=data.iloc[i, p[12]:p[12]+2]  # p12
            )
            
            left_hip_angle[i] = angle_between_points(
                p0=data.iloc[i, p[5]:p[5]+2], # p5
                p1=data.iloc[i, p[11]:p[11]+2], # p11
                p2=data.iloc[i, p[13]:p[13]+2]  # p13
            )
            right_hip_angle[i] = angle_between_points(
                p0=data.iloc[i, p[6]:p[6]+2], # p6
                p1=data.iloc[i, p[12]:p[12]+2], # p12
                p2=data.iloc[i, p[14]:p[14]+2]  # p14
            )

            left_knee_angle[i] = angle_between_points(
                p0=data.iloc[i, p[11]:p[11]+2], # p11
                p1=data.iloc[i, p[13]:p[13]+2], # p13
                p2=data.iloc[i, p[15]:p[15]+2]  # p115
            )
            right_knee_angle[i] = angle_between_points(
                p0=data.iloc[i, p[12]:p[12]+2], # p12
                p1=data.iloc[i, p[14]:p[14]+2], # p14
                p2=data.iloc[i, p[16]:p[16]+2]  # p16
            )
        
        self.df_angle = pd.DataFrame(
            {
                "LEFT_ELBOW_angle": left_elbow_angle,
                "RIGHT_ELBOW_angle": right_elbow_angle,
                "LEFT_ARM_angle": left_arm_angle,
                "RIGHT_ARM_angle": right_arm_angle,
                "LEFT_HIP_angle": left_hip_angle,
                "RIGHT_HIP_angle": right_hip_angle,
                "LEFT_KNEE_angle": left_knee_angle,
                "RIGHT_KNEE_angle": right_knee_angle,
                "class_no": result.to_list()
            }
        )
        pass
    
    def __getitem__(self, idx):
        
        return feature, result
    
    def __len__(self):
        return len(self.dataframe)

In [84]:
ds = GestureDataset(data)

In [85]:
ds.df_angle.head(10)

,LEFT_ELBOW_angle,RIGHT_ELBOW_angle,LEFT_ARM_angle,RIGHT_ARM_angle,LEFT_HIP_angle,RIGHT_HIP_angle,LEFT_KNEE_angle,RIGHT_KNEE_angle,class_no
0,30.642375,114.768470,15.373363,19.755488,171.633813,8.652292,1.065169,175.890112,0
1,29.499736,116.414405,16.151436,19.547796,171.646912,8.574907,1.235083,176.824857,0
2,28.517474,115.925517,15.169092,19.285775,170.834606,8.672581,1.747233,176.371468,0
3,26.177268,115.770147,15.046427,19.970925,171.150329,8.860190,1.969037,175.888648,0
4,31.196002,117.524903,16.660784,19.724086,171.368384,8.962361,1.737987,175.859401,0
5,24.674076,110.481907,14.309520,19.828555,171.506014,8.281219,1.999915,176.345542,0
6,26.922999,110.682096,14.558749,20.287753,171.977464,8.453597,0.810897,176.590451,0
7,22.723897,108.153169,13.714592,20.598305,170.323240,9.404474,1.769444,175.853615,0
8,30.474724,113.388003,16.015997,20.415172,171.644809,9.384299,1.589578,176.111409,0
9,31.033107,110.916889,16.736599,20.769492,171.940160,8.367134,2.303304,175.865866,0


In [9]:
# 姿態模型
class GestureModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.relu_linear_stack = nn.Sequential(
            nn.Linear(34, 68),
            nn.Dropout(p=0.1),
            nn.ReLU(),
            nn.Linear(68, 4)
        )
        pass
    
    def forward(self, x):
        logits = self.relu_linear_stack(x)
        return logits

In [8]:
# # 姿態資料集(已棄用)
# class GestureDataset(Dataset):
#     def __init__(self, df:pd.DataFrame):
#         self.dataframe = df
#         pass
    
#     def __getitem__(self, idx):
#         # 取得資料特徵，轉換成 torch 格式，資料為 torch.float32 格式
#         feature = torch.tensor(self.dataframe.iloc[idx, :-1]).to(torch.float32)
#         # 取得結果
#         result = self.dataframe.iloc[idx, -1]
#         return feature, result
    
#     def __len__(self):
#         return len(self.dataframe)

# 5. 使用資料

In [13]:
# 讀取 csv 資料
gesture_df = pd.read_csv(gesture_data_file)

# 切分成訓練以及測試資料
train_df = gesture_df.iloc[::3]
test_df = gesture_df.iloc[1::2]

# 建立資料集
train_dataset = GestureDataset(train_df)
test_dataset = GestureDataset(test_df)

# 建立加載器
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# 6. 建立模型

In [14]:
# 建立模型
model = GestureModel().to(device)
print(model)

GestureModel(
  (relu_linear_stack): Sequential(
    (0): Linear(in_features=34, out_features=68, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=68, out_features=4, bias=True)
  )
)


# 7. 訓練模型

In [15]:
# 訓練模型
optimizer = optim_fn(model.parameters(), lr=learning_rate)

for t in range(epochs):
    # print(f"Epoch {t+1}\n-------------------------------")
    # 訓練模型
    train(train_dataloader, model, loss_fn, optimizer)
    # 驗證模型(使用訓練資料)
    train_accuracy, train_loss = test(train_dataloader, model, loss_fn)
    # 驗證模型(使用驗證資料)
    # test_accuracy, test_loss = test(test_dataloader, model, loss_fn)
    
    # 畫出折線圖
    writer.add_scalar('Accuracy/train', train_accuracy, t)
    writer.add_scalar('Loss/train', train_loss, t)
    # writer.add_scalar('Accuracy/test', test_accuracy, t)
    # writer.add_scalar('Loss/test', test_loss, t)
    
    writer.flush() # 上傳資料到 TensorBoard

# 繪畫出模型圖
writer.add_graph(model, train_dataset[0][0].to(device))

writer.flush() # 上傳資料到 TensorBoard
writer.close() # 關閉 TensorBoard 的上傳

# PlaySound() # 完成訓練後撥放音樂
print("Done!")

Done!


In [22]:
print(model(train_dataset[0][0].to(device)).argmax(0))

tensor(0, device='cuda:0')
